In [107]:
import numpy as np
import pandas as pd
#import sklearn
from itertools import combinations_with_replacement,chain,permutations, product
from nltk import ngrams
import os
import re

Read from directory and creating dataframe

In [34]:
alphabet = 'абвгдеёжзийклмнопрстуфхцчьыъшщэюя'

In [130]:
def read_directory(dirname):
    array = []
    for _dir in os.listdir(dirname):
        for file in os.listdir(os.path.join(dirname,_dir)):
            filepath = os.path.join(dirname,_dir,file)
            f = open(filepath, 'r')
            text = f.read()
            f.close()
            authors_features = extract_features(text)
            #text_grams = []
            #for i in range(2,3):
            authors_features.update(set_text_data_row(authors_features['only_letters'],alphabet,2))
            authors_features.update(set_text_data_row(authors_features['only_letters'],alphabet,3))
            authors_features.update({'name_author' : _dir})
            #print(authors_features)
            array.append(authors_features)
    return array
arrays = read_directory('textes')
df = pd.DataFrame(arrays)
df

,!/paragraph,",/paragraph",./paragraph,?/paragraph,count_letters,count_paragraphs,name_author,only_letters,symbols/paragraph,аа,...,ёёч,ёёш,ёёщ,ёёъ,ёёы,ёёь,ёёэ,ёёю,ёёя,ёёё
0,0.000000,0.860465,0.000000,0.069767,1216,43,guf,наединственнойизвестнойживойпланетенасамомболь...,34.930233,0,...,0,0,0,0,0,0,0,0,0,0
1,0.097826,0.608696,0.054348,0.239130,2371,92,guf,давайтенемногоподумаемпосмеемсяоднусекундочкус...,32.630435,0,...,0,0,0,0,0,0,0,0,0,0
2,0.000000,0.860465,0.302326,0.069767,1340,43,guf,явообщетонепривыкжаловатьсяназдоровьеикомукако...,39.534884,0,...,0,0,0,0,0,0,0,0,0,0
3,0.000000,0.462963,0.000000,0.000000,1578,54,guf,когдамнеоченьплохоилислишкомхорошояделаювздохи...,36.185185,0,...,0,0,0,0,0,0,0,0,0,0
4,0.000000,1.013333,0.213333,0.000000,2037,75,guf,всекакмылюбимэтопростомысливслухкогдапотухнетс...,34.400000,0,...,0,0,0,0,0,0,0,0,0,0
5,0.000000,0.693333,0.360000,0.000000,1753,75,noizemc,затянисьмноювпоследнийразткнименямордойвстекло...,29.733333,0,...,0,0,0,0,0,0,0,0,0,0
6,0.000000,0.764706,0.397059,0.000000,1748,68,noizemc,есливселеннаябесконечнадействительнозначитесть...,32.367647,0,...,0,0,0,0,0,0,0,0,0,0
7,0.000000,0.409091,0.000000,0.000000,1229,44,noizemc,моёморепрошутебяневыплюнименянаберегвовремяоче...,34.840909,0,...,0,0,0,0,0,0,0,0,0,0
8,0.000000,1.394737,0.355263,0.000000,2028,76,noizemc,лицабородатыекамуфляжавтоматынеприятныеребятав...,34.394737,1,...,0,0,0,0,0,0,0,0,0,0
9,0.000000,0.549020,0.274510,0.000000,1634,51,noizemc,намарсекласснокрасныепескикрутыегорыикратерыбе...,39.490196,0,...,0,0,0,0,0,0,0,0,0,0


In [115]:
def extract_features(text):
    count_paragraphs = text.count('\n')+1
    striped_text = text.strip()
    only_letters = ''.join(filter(str.isalpha, text))
    #print(only_letters)
    return {
        'symbols/paragraph' : len(striped_text)/count_paragraphs,
        './paragraph' : striped_text.count('.')/count_paragraphs,
        ',/paragraph' : striped_text.count(',')/count_paragraphs,
        '!/paragraph' : striped_text.count('!')/count_paragraphs,
        '?/paragraph' : striped_text.count('?')/count_paragraphs,
        'count_paragraphs' : count_paragraphs,
        'count_letters' : len(only_letters),
        'only_letters' : only_letters.lower()
    }

In [ ]:
def get_letter_combination(alphabet, count):
    return [''.join(val) for val in [p for p in product(list(alphabet),repeat = count)]]

In [118]:
def set_text_data_row(text, alphabet, n):
    text = text.lower()
    text_grams = [''.join(i) for i in ngrams(text, n)]
    combinations = get_letter_combination(alphabet, n)
    res = {}
    for i in combinations:
        res[i] = 0
        for j in text_grams:
            if j == i:
                res[i] += 1
    return res